In [17]:
from dotenv import dotenv_values
import requests
import json
import random
import numpy as np
import pickle
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
en_stopwords = stopwords.words('english')

config = dotenv_values("../.env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}
debug_server = config['DEBUG_SERVER']
token = config['TOKEN']
submitt_url = config['DEBUG_SERVER_SUBMIT']
HEADER = {'content-type': 'application/json'}

# ML Model

In [2]:
def remove_whitespace(text):
   """ Removes all whitespaces from a given text."""
   return " ".join(text.split())

def remove_URLs(text):
   "Remove URLs from text using regular expressions."
   url_re = re.compile(r'https?://\S+|www\.\S+')
   return url_re.sub(r'', text)

def remove_punctuation(text):
   tokenizer = RegexpTokenizer(r'\w+')
   no_punct = tokenizer.tokenize(" ".join(text))
   return no_punct

def remove_stopwords(text):
   """Removes english stopwords."""
   result = []
   for token in text:
      if token not in en_stopwords:
         result.append(token)

   return result

def stemming(text):
   porter = PorterStemmer()
   result = []
   for word in text:
      result.append(porter.stem(word))

   return result

In [5]:
def ml_text_processing(text):
    text = text.lower()
    text = remove_whitespace(text)
    text = remove_URLs(text)
    text = word_tokenize(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = stemming(text)
    return text

def ml_predict_pipeline(text, path):
    text = ml_text_processing(text)
    loaded_model = pickle.load(open(path, 'rb'))
    return loaded_model.predict(text)

In [8]:
text = "Hello world how are you?"
path = "../models/ml_models/m1_LR_CV.sav"
model = pickle.load(open(path, 'rb'))

In [11]:
p_text = ml_text_processing(text)
print(p_text)

['hello', 'world']


In [18]:
model.predict(np.array([p_text]))

/home/tudor/.mamba/envs/pytorch/lib/python3.8/site-packages/sklearn/base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


ValueError: Unable to convert array of bytes/strings into decimal numbers with dtype='numeric'

## DL Model

In [28]:
from data import DataModule
from models import Transformer
from transformers import RobertaTokenizerFast
import torch
import numpy as np

model_path = r"../models/transformer/model_val_loss0.28.ckpt"
tokenizer_path =  r"../models/roberta-tokenizer"

In [29]:
system1 = Transformer(
    ntokens=30000,
    emsize=128,
    d_hid=128,
    nlayers=1,
    nhead=2,
    dropout=0.2,
).load_from_checkpoint(model_path)
system1.eval()
system1_tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)


file ../models/roberta-tokenizer/config.json not found
file ../models/roberta-tokenizer/config.json not found


In [33]:
def get_prediction(text):
    text = system1_tokenizer.encode_plus(
    text,
    # add_special_tokens=True,
    max_length=512,
    padding="max_length",
    return_tensors="pt",
    return_attention_mask=False,
    truncation=True,
    )
    output = torch.sigmoid(system1(text['input_ids'])).detach().numpy()
    return np.heaviside(output, 0)

# Official Submission

## Workflow
```
getWritings
while request not empty:
    for run in runs:
        POST request
    getWritings
```

In [23]:
GET_URL = 'http://127.0.0.1:5000'
systems = [get_prediction, None, None, None, None]
ans = requests.get(GET_URL)
ans_dict = json.loads(ans.text)
while ans_dict != []:
    for run, system in enumerate(systems):
        post_url = config['T1_SUBMISSION'] + token + "/" + str(run)
        submission = []
        for ans in ans_dict:
            nick = ans['nick']
            text = ans['content']
            submission.append({
                'nick': nick,
                'decision': system(text),
                'score': round(random.uniform(0,4),1)
            })
        post_request = requests.post(post_url, data = json.dumps(submission), headers = HEADER)
    ans = requests.get(GET_URL)
    ans_dict = json.loads(ans.text)

[]
